In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from collections import namedtuple

## Define a planet class

In [20]:
class planet():
    "A planet in our solar system"
    def __init__(self,semimajor,eccentricity):
        self.x = np.zeros(2)                   # x and y positions
        self.v = np.zeros(2)                   # x and y velocity
        self.a_g = np.zeros(2)                 # x and y acceleration
        self.t = 0.0                           # current time
        self.dt = 0.0                          # current timestep
        self.a = semimajor                     # semimajor axis of the orbit
        self.e = eccentricity                  # eccentricity of the orbit
        self.istep = 0                         # current integer timestep
        self.name = ""                         # name for our planet

## Define a dictionary with some constants

In [21]:
solar_system = {"M_sun":1.0, "G":39.4784176043574320}

## Define some functions for setting circular velocity, and acceleration

In [22]:
def SolarCircularVelocity(p):
    
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = (p.x[0]**2 + p.x[1]**2)**0.5
    
    #return the circular velocity
    return (G*M/r)**0.5

## Write a function to compute the gravitational acceleration on each planet from the Sun

In [23]:
def SolarGravitationalAcceleration(p):
    
    G = solar_system["G"]
    M = solar_system["M"]
    r = (p.x[0]**2 + p.x[1]**2)**0.5
    
    # Acceleration in AU/yr/yr
    a_grav = -1.0*G*M/r**2
    
    # Find the angle at this position
    if(p.x[0]==0.0):
        if(p.x[1]>0.0):
            theta = 0.5*np.pi
        else:
            theta = 1.5*np.pi
    else:
        theta = np.arctan2(p.x[1],p.x[0])
        
    # Set the x and y components of the gravity
    # p.a_g[0] = a_grav * np.cos(theta)
    # p.a_g[1] = a_grav * np.sin(theta)
    return a_grav*np.cos(theta), a_grav*np.sin(theta)

## Compute the timestep

In [24]:
def calc_dt(p):
    
    # Integration tolerance
    ETA_TIME_STEP = 0.0004
    
    # Compute the timestep
    eta = ETA_TIME_STEP
    v = (p.v[0]**2 + p.v[1]**2)**0.5
    a = (p.a_g[0]**2 + p.a_g[1]**2)**0.5
    dt = eta*np.fmin(1./np.fabs(v),1./np.fabs(a)**0.5)
    
    return dt

## Define the initial conditions

In [25]:
def SetPlanet(p,i):
    
    AU_in_km = 1.495979e8        # An AU in km
    
    # Circular velocity
    v_c = 0.0                    # Circular velocity in AU/yr
    v_e = 0.0                    # Velocity at perihelion in AU/yr
    
    # Planet-by-planet initial conditions
    
    # Mercury
    if(i==0):
        # Semi-major axis in AU
        p.a = 57909227.0/AU_in_km
        
        # Eccentricity
        p.e = 0.20563593
        
        # Name
        p.name = "Mercury"
        
    # Venus
    elif(i==1):
        # Semi-major axis in AU
        p.a = 108209475.0/AU_in_km
        
        # Eccentricity
        p.e = 0.00677672
        
        # Name
        p.name = "Venus"
    
    # Earth
    elif(i==2):
        # Semi-major axis in AU
        p.a = 1.0
        
        # Eccentricity
        p.e = 0.01671123
        
        # Name
        p.name = "Earth"
        
    # Set remaining properties
    p.t = 0.0
    p.x[0] = p.a*(1.0-p.e)
    p.x[1] = 0.0
    
    # Get equiv circular velocity
    v_c = SolarCircularVelocity(p)
    
    # Velocity at perihelion
    v_e = v_c*(1 + p.e)**0.5
    
    # Set velocity
    p.v[0] = 0.0                    # No x velocity at perihelion
    p.v[1] = v.e                    # y velocity at perihelion (counter-clockwise)
    
    # Calculate gravitational acceleration from the sun
    p.a_g = SolarGravitationalAcceleration(p)
    
    # Set timestep
    p.dt = calc_dt(p)

## Write leapfrog integrator

In [26]:
def x_first_step(x_i,v_i,a_i,dt):
    #x1/2 = x_0 + 1/2 v_0 Delta_t + 1/4 a_0 Delta t^2
    return x_i + 0.5*v_i*dt + 0.25*a_i*dt**2

In [27]:
def v_full_step(v_i, a_ipoh, dt):
    #v_i+1 = v_i + a_i+1/2 Delta t
    return v_i + a_ipoh*dt

In [28]:
def x_full_step(x_ipoh,v_ip1, a_ipoh, dt):
    #x_3/2 = x_1/2 + v_i+1 Delta t
    return x+ipoh + v_ip1*dt

## Write a function to save the data to a file

In [29]:
def SaveSolarSystem(p, n_planets, t, dt, istep, ndim):
    
    # Loop over the number of planets
    for i in range(n_planets):
        
        # Define a filename
        fname = "planet.%s.txt" % p[i].name
        
        if(istep==0):
            # Create the file on the first timestep
            fp = open(fname,"w")
        else:
            # Append the file on subsequent timesteps
            fp = open(fname,"a")
            
        # Compute the drifted properties of the planet
        v_drift = np.zeros(ndim)
        
        for k in range(ndim):
            v_drift[k] = p[i].v[k] + 0.5*p[i].a_g[k]*p[i].dt
            
        # Write data to the file
        s = "%6d\t%6.5f\t%6.5\t%6d\t%6.5f\t%6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\n" % \
        (istep,t,dt,p[i].istep,p[i].t,p[i].dt,p[i].x[0],p[i].x[1],v_drift[0],v_drift[1],              \
         p[i].a_g[0],p[i].a_g[1])
        fp.write(s)
        
        # Close the file
        fp.close()

## Write a function to evolve the solar system

In [30]:
def EvolveSolarSystem(p, n_planets, t_max):
    
    # Number of spatial dimensions
    ndim = 2
    
    # Define the first timestep
    dt = 0.5/365.25
    
    # Define the starting time
    t = 0
    
    # Define the starting timestep
    istep = 0
    
    # Save the initial conditions
    SaveSolarSystem(p,n_planets,t,dt,istep,ndim)
    
    # Begin a loop over the global timescale
    while(t>tmax):
        
        # Check to see if the next step exceeds the
        # maximum time. If so, take a smaller step
        if(t+dt>t_max):
            dt = t_max - t                      #limit the step to align with t_max
            
        # Evolve each planet
        for i in range(n_planets):
            
            while(p[i].t<t+dt):
                
                # Special case for istep==0
                if(p[i].istep==0):
                    
                    # Take the first step according to a verlet scheme
                    for k in range(ndim):
                        p[i].x[k] = x_first_step(p[i].x[k],p[i].v[k],p[i].a_g[k],p[i].dt)
                        
                    # Update the acceleration
                    p[i].a_g = SolarGravitationalAcceleration(p[i])
                    
                    # Update the time by 1/2dt
                    p[i].t += 0.5*p[i].dt
                    
                    # Update the timestep
                    p[i].dt = calc_dt(p[i])
                    
                # Continue with a normal step
                
                # Limit to align with the global timestep
                if(p[i].t + p[i].dt > t+dt):
                    p[i].dt = t+dt-p[i].t
                    
                # Evolve the velocity
                for k in range(ndim):
                    p[i].v[k] = v_full_step(p[i].v[k],p[i].a_g[k],p[i].dt)
                
                # Evolve the position
                for k in range(ndim):
                    p[i].x[k] = x_full_step(p[i].x[k],p[i].v[k],p[i].a_g[k],p[i].dt)
                    
                # Update the acceleration
                p[i].a_g = SolarGravitationalAcceleration(p[i])
                
                # Update for dt
                p[i].t += p[i].dt
                
                # Compute the new timestep
                p[i].dt = calc_dt(p[i])
                
                # Update the planet's timestep
                p[i].istep += 1
                
            # Now update the global system time
            t += dt
            
            # Update the global step number
            istep += 1
            
            # Output the current step
            SaveSolarSystem(p,n_planets,t,dt,istep,ndim)
            
        # Print the final steps and time
        print("Time t = ",t)
        print("Maximum t = ",t_max)
        print("Maximum number of steps = ",istep)
        
        # End the evolution

## Create a routine to read in the data

In [31]:
def read_twelve_arrays(fname):
    fp = open(fname,"r")
    fl = fp.readlines()
    n = len(fl)
    a = np.zeros(n)
    b = np.zeros(n)
    c = np.zeros(n)
    d = np.zeros(n)
    f = np.zeros(n)
    g = np.zeros(n)
    h = np.zeros(n)
    j = np.zeros(n)
    k = np.zeros(n)
    l = np.zeros(n)
    m = np.zeros(n)
    p = np.zeros(n)
    for i in range(n):
        a[i] = float(fl[i].split()[0])
        b[i] = float(fl[i].split()[1])
        c[i] = float(fl[i].split()[2])
        d[i] = float(fl[i].split()[3])
        f[i] = float(fl[i].split()[4])
        g[i] = float(fl[i].split()[5])
        h[i] = float(fl[i].split()[6])
        j[i] = float(fl[i].split()[7])
        k[i] = float(fl[i].split()[8])
        l[i] = float(fl[i].split()[9])
        m[i] = float(fl[i].split()[10])
        p[i] = float(fl[i].split()[11])
        
    return a,b,c,d,f,g,h,j,k,l,m,p

## Perform the integration of the solar system